# Lab 3-2: E2E ML Pipeline (Part 1)

이 노트북에서는 **End-to-End ML Pipeline**을 구축합니다.

## 학습 목표

1. Kubeflow Pipeline 구성요소 이해
2. MLflow 연동
3. 조건부 배포
4. KServe 배포

---

## 1. 환경 설정

In [ ]:
# 패키지 설치
!pip install kfp>=2.0.0 mlflow>=2.9.0 scikit-learn pandas numpy -q
print("✅ 패키지 설치 완료!")

In [ ]:
# Import
import os
from kfp import dsl
from kfp.dsl import component, Input, Output, Dataset, Model
from kfp import compiler
import kfp

print(f"KFP Version: {kfp.__version__}")

In [ ]:
# 환경 변수 설정
USER_NUM = "01"  # TODO: 본인 번호로 변경!
USER_NAMESPACE = f"kubeflow-user{USER_NUM}"

MLFLOW_TRACKING_URI = os.getenv(
    'MLFLOW_TRACKING_URI',
    'http://mlflow-server-service.mlflow-system.svc.cluster.local:5000'
)

print(f"User Number: {USER_NUM}")
print(f"Namespace: {USER_NAMESPACE}")
print(f"MLflow URI: {MLFLOW_TRACKING_URI}")

---

## 2. 컴포넌트 정의

### 2.1 데이터 로드

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2"]
)
def load_data(data_source: str, output_data: Output[Dataset]):
    """California Housing 데이터셋 로드"""
    import pandas as pd
    from sklearn.datasets import fetch_california_housing
    
    print("=" * 50)
    print("  Step 1: Load Data")
    print("=" * 50)
    
    if data_source == "sklearn":
        housing = fetch_california_housing(as_frame=True)
        df = housing.frame
    else:
        df = pd.read_csv(data_source)
    
    print(f"  Data shape: {df.shape}")
    df.to_csv(output_data.path, index=False)
    print(f"  ✅ Saved to: {output_data.path}")

### 2.2 전처리

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2", "numpy==1.24.3"]
)
def preprocess(
    input_data: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset],
    y_train_out: Output[Dataset],
    y_test_out: Output[Dataset],
    test_size: float = 0.2
):
    """데이터 전처리: Train/Test 분할 및 정규화"""
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    print("=" * 50)
    print("  Step 2: Preprocess")
    print("=" * 50)
    
    df = pd.read_csv(input_data.path)
    
    X = df.drop(columns=['MedHouseVal'])
    y = df['MedHouseVal']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )
    
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train),
        columns=X_train.columns
    )
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test),
        columns=X_test.columns
    )
    
    X_train_scaled.to_csv(X_train_out.path, index=False)
    X_test_scaled.to_csv(X_test_out.path, index=False)
    y_train.to_csv(y_train_out.path, index=False)
    y_test.to_csv(y_test_out.path, index=False)
    
    print(f"  Train: {len(X_train)}, Test: {len(X_test)}")
    print(f"  ✅ Preprocessing completed")

### 2.3 피처 엔지니어링

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "numpy==1.24.3"]
)
def feature_engineering(
    X_train_in: Input[Dataset],
    X_test_in: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset]
):
    """피처 엔지니어링: 파생 변수 생성"""
    import pandas as pd
    import numpy as np
    
    print("=" * 50)
    print("  Step 3: Feature Engineering")
    print("=" * 50)
    
    X_train = pd.read_csv(X_train_in.path)
    X_test = pd.read_csv(X_test_in.path)
    
    def add_features(df):
        df = df.copy()
        df['rooms_per_household'] = df['AveRooms'] / (df['AveOccup'] + 1e-6)
        df['bedrooms_ratio'] = df['AveBedrms'] / (df['AveRooms'] + 1e-6)
        df['population_per_household'] = df['Population'] / (df['AveOccup'] + 1e-6)
        return df
    
    X_train_fe = add_features(X_train)
    X_test_fe = add_features(X_test)
    
    X_train_fe.to_csv(X_train_out.path, index=False)
    X_test_fe.to_csv(X_test_out.path, index=False)
    
    print(f"  Features: {len(X_train_fe.columns)}")
    print(f"  ✅ Feature engineering completed")

### 2.4 모델 학습 (MLflow 연동)

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=[
        "pandas==2.0.3",
        "scikit-learn==1.3.2",
        "mlflow==2.9.2",
        "numpy==1.24.3"
    ]
)
def train_model(
    X_train: Input[Dataset],
    X_test: Input[Dataset],
    y_train: Input[Dataset],
    y_test: Input[Dataset],
    mlflow_tracking_uri: str,
    experiment_name: str,
    n_estimators: int = 100,
    max_depth: int = 10
) -> str:
    """모델 학습 및 MLflow 기록"""
    import pandas as pd
    import numpy as np
    import mlflow
    import mlflow.sklearn
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import os
    
    print("=" * 50)
    print("  Step 4: Train Model")
    print("=" * 50)
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    
    X_train_df = pd.read_csv(X_train.path)
    X_test_df = pd.read_csv(X_test.path)
    y_train_df = pd.read_csv(y_train.path)
    y_test_df = pd.read_csv(y_test.path)
    
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run() as run:
        run_id = run.info.run_id
        
        params = {"n_estimators": n_estimators, "max_depth": max_depth, "random_state": 42}
        mlflow.log_params(params)
        
        model = RandomForestRegressor(**params)
        model.fit(X_train_df, y_train_df.values.ravel())
        
        y_pred = model.predict(X_test_df)
        
        r2 = r2_score(y_test_df, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_df, y_pred))
        mae = mean_absolute_error(y_test_df, y_pred)
        
        mlflow.log_metrics({"r2": r2, "rmse": rmse, "mae": mae})
        mlflow.sklearn.log_model(model, "model", registered_model_name=f"{experiment_name}-model")
        
        print(f"  R2: {r2:.4f}, RMSE: {rmse:.4f}")
        print(f"  ✅ Run ID: {run_id}")
    
    return run_id

### 2.5 모델 평가

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["mlflow==2.9.2"]
)
def evaluate_model(
    run_id: str,
    mlflow_tracking_uri: str,
    r2_threshold: float = 0.75
) -> str:
    """모델 평가 및 배포 결정"""
    import mlflow
    import os
    
    print("=" * 50)
    print("  Step 5: Evaluate Model")
    print("=" * 50)
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    
    client = mlflow.tracking.MlflowClient()
    run = client.get_run(run_id)
    
    r2 = float(run.data.metrics.get("r2", 0))
    
    print(f"  R2: {r2:.4f}, Threshold: {r2_threshold}")
    
    if r2 >= r2_threshold:
        decision = "deploy"
        print(f"  ✅ Decision: DEPLOY")
    else:
        decision = "skip"
        print(f"  ⚠️ Decision: SKIP")
    
    return decision

### 2.6 모델 배포 / 알림

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["kubernetes==28.1.0"]
)
def deploy_model(run_id: str, model_name: str, namespace: str):
    """KServe 배포"""
    from kubernetes import client, config
    import time
    
    print("=" * 50)
    print("  Step 6: Deploy Model")
    print("=" * 50)
    
    print(f"  Model: {model_name}")
    print(f"  Namespace: {namespace}")
    print(f"  Run ID: {run_id}")
    print(f"  ✅ Deployment initiated")


@component(base_image="python:3.9-slim")
def send_alert(run_id: str, message: str = "Model did not meet threshold"):
    """알림 전송"""
    print("=" * 50)
    print("  Alert")
    print("=" * 50)
    print(f"  ⚠️ {message}")
    print(f"  Run ID: {run_id}")

---

## 3. 파이프라인 정의

In [ ]:
@dsl.pipeline(
    name="E2E ML Pipeline",
    description="End-to-End Machine Learning Pipeline"
)
def e2e_ml_pipeline(
    data_source: str = "sklearn",
    experiment_name: str = "e2e-pipeline",
    model_name: str = "california-model",
    namespace: str = "kubeflow-user01",
    mlflow_tracking_uri: str = "http://mlflow-server-service.mlflow-system.svc.cluster.local:5000",
    n_estimators: int = 100,
    max_depth: int = 10,
    r2_threshold: float = 0.75
):
    # Step 1: 데이터 로드
    load_task = load_data(data_source=data_source)
    
    # Step 2: 전처리
    preprocess_task = preprocess(input_data=load_task.outputs["output_data"])
    
    # Step 3: 피처 엔지니어링
    feature_task = feature_engineering(
        X_train_in=preprocess_task.outputs["X_train_out"],
        X_test_in=preprocess_task.outputs["X_test_out"]
    )
    
    # Step 4: 모델 학습
    train_task = train_model(
        X_train=feature_task.outputs["X_train_out"],
        X_test=feature_task.outputs["X_test_out"],
        y_train=preprocess_task.outputs["y_train_out"],
        y_test=preprocess_task.outputs["y_test_out"],
        mlflow_tracking_uri=mlflow_tracking_uri,
        experiment_name=experiment_name,
        n_estimators=n_estimators,
        max_depth=max_depth
    )
    
    # Step 5: 평가
    evaluate_task = evaluate_model(
        run_id=train_task.output,
        mlflow_tracking_uri=mlflow_tracking_uri,
        r2_threshold=r2_threshold
    )
    
    # Step 6: 조건부 배포
    with dsl.If(evaluate_task.output == "deploy"):
        deploy_model(
            run_id=train_task.output,
            model_name=model_name,
            namespace=namespace
        )
    
    with dsl.If(evaluate_task.output == "skip"):
        send_alert(run_id=train_task.output)

print("✅ 파이프라인 정의 완료!")

---

## 4. 파이프라인 컴파일

In [ ]:
pipeline_file = "e2e_pipeline.yaml"

compiler.Compiler().compile(
    pipeline_func=e2e_ml_pipeline,
    package_path=pipeline_file
)

print(f"✅ Pipeline compiled: {pipeline_file}")
print(f"\n다음 단계:")
print(f"  1. Kubeflow UI → Pipelines → Upload pipeline")
print(f"  2. {pipeline_file} 파일 선택")
print(f"  3. Create Run → Parameters 설정 → Start")

---

## 5. 파이프라인 실행 (선택)

In [ ]:
# KFP Client로 직접 실행 (클러스터 내부에서만 동작)
try:
    client = kfp.Client()
    print(f"✅ KFP Client connected")
    print(f"   Host: {client._host}")
    
    run = client.create_run_from_pipeline_func(
        e2e_ml_pipeline,
        arguments={
            'data_source': 'sklearn',
            'experiment_name': f'e2e-pipeline-{USER_NUM}',
            'model_name': f'california-model-{USER_NUM}',
            'namespace': USER_NAMESPACE,
            'n_estimators': 100,
            'max_depth': 10,
            'r2_threshold': 0.75
        },
        experiment_name=f'e2e-experiment-{USER_NUM}',
        run_name=f'e2e-run-{USER_NUM}-001'
    )
    
    print(f"\n✅ Pipeline submitted!")
    print(f"   Run ID: {run.run_id}")
    
except Exception as e:
    print(f"⚠️ Could not connect to KFP: {e}")
    print(f"\n💡 Upload YAML manually via Kubeflow UI")

---

## ✅ Part 1 완료!

### 학습한 내용
- Kubeflow Pipeline 컴포넌트 정의
- MLflow 연동
- 조건부 분기 (`dsl.If`)
- 파이프라인 컴파일 및 실행

### 다음 단계
- Part 2: 조별 프로젝트 실습
- Part 3: 발표 및 피드백